In [3]:
# import sys

# sys.path.append("/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/src/script")
# from DEFsv1-Cylinder import *

In [1]:
import commands
import numpy as np
import nibabel as nib
import os
from sklearn import linear_model
from scipy import stats
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import subprocess
from skimage import data, img_as_float, color, exposure
from skimage.restoration import unwrap_phase
import loopfield as lf
import pandas as pd

In [2]:
def theory_map(sub_path, position, normal, radius, current, bp):
    field = lf.Field(length_units = lf.cm, current_units = lf.A, field_units = lf.uT)
    # single-turn 12 cm z-oriented coil at origin

    c = lf.Loop(position, normal, radius, current)
    # add loop to field
    field.addLoop(c);
    #MAKE THEORETICAL B FIELD MAPS
    #read in xyz ijk coordinates from i=0 masked image
    cwd_path = sub_path+"/NIFTI/bmaps_numpy/"
    for i in range(0,len(bp)):
        print i
        df_t=pd.DataFrame()
        Bvals = list()
        df_t_geo = pd.DataFrame()
        df_Bvals = pd.DataFrame()
        bdir = cwd_path+bp[i][0]+'minus'+bp[i][1]+"/"
        print bdir
        #make mask
        commands.getoutput("fslmaths "+bdir+"4d_fmap_Mean.nii.gz -div "+bdir+"4d_fmap_Mean.nii.gz "+bdir+"mask.nii.gz")
        commands.getoutput("3dmaskdump -o "+bdir+"/mask.txt -xyz -nozero "+bdir+"mask.nii.gz")
        df_t = pd.read_csv(bdir+"/mask.txt", sep=" ", header=None)
        a = len(df_t)
        for j in range(0,a-1):
            bval = 1000*field.evaluate([float(df_t[3][j])/10.,float(df_t[4][j])/10.,float(df_t[5][j])/10.])
            Bvals.append(bval[2])

        
        k = i%3
        if k ==0:
            print k
            df_t_geo = df_t
            df_t_geo = df_t_geo.drop(6,1)
            df_t_geo = df_t_geo.drop(5,1)
            df_t_geo = df_t_geo.drop(4,1)
            df_t_geo = df_t_geo.drop(3,1)
            Bvals_col = pd.Series(Bvals)
            df_Bvals = df_t_geo
            df_Bvals.insert(loc=3,column = 3,value=Bvals_col)
            df_Bvals.to_csv(bdir+"/Bvals.txt", header=None, index=None, sep=' ')
        #!!!!! here you use a different -master and -mask file for 3dundump--need to fix that to all be the same
            commands.getoutput("3dUndump -ijk -datum float -prefix "+bdir+"/Bvals.nii -master "+bdir+"/mask.nii.gz -mask "+bdir+"mask.nii.gz "+bdir+"/Bvals.txt")
        if k ==1:
            print k
            Bvals1 = [-1*Bvals[k] for k in range(0,len(Bvals))]
            Bvals1_col = pd.Series(Bvals1)
            df_t_geo = df_t
            df_t_geo = df_t_geo.drop(6,1)
            df_t_geo = df_t_geo.drop(5,1)
            df_t_geo = df_t_geo.drop(4,1)
            df_t_geo = df_t_geo.drop(3,1)
            df_Bvals1 = pd.DataFrame()
            df_Bvals1 = df_t_geo
            df_Bvals1.insert(loc=3,column = 3,value=Bvals1_col)
            df_Bvals1.to_csv(bdir+"/Bvals.txt", header=None, index=None, sep=' ')
        #!!!!! here you use a different -master and -mask file for 3dundump--need to fix that to all be the same
            commands.getoutput("3dUndump -ijk -datum float -prefix "+bdir+"/Bvals.nii -master "+bdir+"/mask.nii.gz -mask "+bdir+"mask.nii.gz "+bdir+"/Bvals.txt")
        if k ==2:
            print k
            Bvals2 = [2*Bvals[k] for k in range(0,len(Bvals))]
            Bvals2_col = pd.Series(Bvals2)
            df_t_geo = df_t
            df_t_geo = df_t_geo.drop(6,1)
            df_t_geo = df_t_geo.drop(5,1)
            df_t_geo = df_t_geo.drop(4,1)
            df_t_geo = df_t_geo.drop(3,1)
            df_Bvals2 = pd.DataFrame()
            df_Bvals2 = df_t_geo
            df_Bvals2.insert(loc=3,column = 3,value=Bvals2_col)
            df_Bvals2.to_csv(bdir+"/Bvals.txt", header=None, index=None, sep=' ')
        #!!!!! here you use a different -master and -mask file for 3dundump--need to fix that to all be the same
            commands.getoutput("3dUndump -ijk -datum float -prefix "+bdir+"/Bvals.nii -master "+bdir+"/mask.nii.gz -mask "+bdir+"mask.nii.gz "+bdir+"/Bvals.txt")


In [9]:
##############
# Cylinder 32
##############

sub_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA"
bp = ['6','4'],['10','4'],['6','10'],['16','14'],['20','14'],['16','20']

position = [0.6, -1.5, 0.0] 
normal = [0., 0., 1.]
radius = 6.0
current = 0.00200
theory_map(sub_path, position, normal, radius, current, bp)

0
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA/NIFTI/bmaps_numpy/6minus4/
0
1
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA/NIFTI/bmaps_numpy/10minus4/
1
2
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA/NIFTI/bmaps_numpy/6minus10/
2
3
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA/NIFTI/bmaps_numpy/16minus14/
0
4
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA/NIFTI/bmaps_numpy/20minus14/
1
5
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_32_aqueous_2mA/NIFTI/bmaps_numpy/16minus20/
2


In [11]:
##############
# Cylinder 33
##############

sub_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA"
bp = ['6','4'],['10','4'],['6','10'],['16','14'],['20','14'],['16','20']

position = [0.45, -1.5, -0.8] # z coordinate is not zero
normal = [0., 0., 1.]
radius = 6.0
current = 0.00200
theory_map(sub_path, position, normal, radius, current, bp)

0
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA/NIFTI/bmaps_numpy/6minus4/
0
1
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA/NIFTI/bmaps_numpy/10minus4/
1
2
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA/NIFTI/bmaps_numpy/6minus10/
2
3
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA/NIFTI/bmaps_numpy/16minus14/
0
4
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA/NIFTI/bmaps_numpy/20minus14/
1
5
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_33_aqueous_2mA/NIFTI/bmaps_numpy/16minus20/
2


In [4]:
##############
# Cylinder 36
##############

sub_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous"
bp = ['6','4'],['10','4'],['6','10'],['16','14'],['20','14'],['16','20'],['26','24'],['30','24'],['26','30'],['36','34'],['40','34'],['36','40'],['46','44'],['50','44'],['46','50'],['56','54'],['60','54'],['56','60']

position = [0.053, -1.76, -0.33] 
normal = [0., 0., 1.]
radius = 6.0
current = 0.00200
theory_map(sub_path, position, normal, radius, current, bp)

0
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/6minus4/
0
1
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/10minus4/
1
2
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/6minus10/
2
3
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/16minus14/
0
4
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/20minus14/
1
5
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/16minus20/
2
6
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/26minus24/
0
7
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_36_Cyl_Aqueous/NIFTI/bmaps_numpy/30minus24/
1
8
/Users/chikakoolsen/opt/py

In [5]:
##############
# Cylinder 37
##############

sub_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous"
bp = ['6','4'],['10','4'],['6','10'],['16','14'],['20','14'],['16','20'],['26','24'],['30','24'],['26','30'],['36','34'],['40','34'],['36','40'],['46','44'],['50','44'],['46','50'],['56','54'],['60','54'],['56','60']

position = [0.089, -1.15, -0.33] 
normal = [0., 0., 1.]
radius = 6.0
current = 0.00200
theory_map(sub_path, position, normal, radius, current, bp)

0
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/6minus4/
0
1
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/10minus4/
1
2
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/6minus10/
2
3
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/16minus14/
0
4
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/20minus14/
1
5
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/16minus20/
2
6
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/26minus24/
0
7
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/30minus24/
1
8
/Users/chikakoolsen/opt/py

IOError: [Errno 2] File /Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/56minus54//mask.txt does not exist: '/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_37_Cyl_Aqueous/NIFTI/bmaps_numpy/56minus54//mask.txt'

In [6]:
##############
# Cylinder 38
##############

sub_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous"
bp = ['6','4'],['10','4'],['6','10'],['16','14'],['20','14'],['16','20'],['26','24'],['30','24'],['26','30'],['36','34'],['40','34'],['36','40'],['46','44'],['50','44'],['46','50'],['56','54'],['60','54'],['56','60']

position = [-0.053, -1.33, 0.025] 
normal = [0., 0., 1.]
radius = 6.0
current = 0.00200
theory_map(sub_path, position, normal, radius, current, bp)

0
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/6minus4/
0
1
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/10minus4/
1
2
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/6minus10/
2
3
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/16minus14/
0
4
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/20minus14/
1
5
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/16minus20/
2
6
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/26minus24/
0
7
/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/processed/R21_38_Cyl_Aqueous/NIFTI/bmaps_numpy/30minus24/
1
8
/Users/chikakoolsen/opt/py